## 七月在线电商推荐系统项目特训营结业考试
#### 考试说明:
- 起止时间：<font color=red><b>2020年2月28日10:00 - 2020年2月29日00:00</b></font>，<b>逾期不接受补考</b>
- 考试提交方式：<font color=red><b> 将试卷下载到本地作答</b></font>，本地作答完成后将文件命名为<font color=red><b>"姓名_学号"</b></font>并上传到：http://47.93.208.249:9825/tree/0.Teacher/Exam/Stage1
- 注意事项：为确保同学们真正了解自身对这次课程的掌握程度，<font color=red><b>请勿翻阅抄袭，移动，更改</b></font>其它同学的试卷，如发现按0分处理。
- 请同学在下方同学姓名处 <font color=red><b>填写自己的姓名</b></font>，批改人和最终得分处不用填写，试卷总分为100分。
- 考试用到的数据下载地址：http://47.93.208.249:9825/tree/0.Teacher/Exam ， 将数据 movies.dat 和 ratings.dat 放在与试卷同目录下使用

- 同学姓名:ywuan
- 批改人：魏天保
- 最终得分: 100

<center><h1>####答卷开始####</h1></center>

## 一、理论题（40分）

#### 1、协同过滤算法是指基于用户行为数据设计的推荐算法，常见的有UserCF和ItemCF两种，请你分别用一句话概括这两种算法的核心思想。（10分）

In [ ]:
UserCF的核心思想是：给用户推荐和他兴趣相似的其他用户喜欢的物品。ItemCF的核心思想是：给用户推荐和其过去感兴趣的物品相似的物品。

#### 2、 矩阵分解属于协同过滤算法吗？（5分）

A 属于

B 不属于

In [ ]:
A

#### 3、请列举出基于内容的推荐的两个优点（6分）

In [ ]:
1、不需要用户的历史数据，如对对象的评价等。
2、没有关于新推荐对象出现的冷启动问题。

#### 4、请说出推荐系统中评分预测和TopN推荐的评价指标，各列举两种。（4分）

In [ ]:
评分预测：RMSE、MSE
TopN推荐：Precision、Recall。

#### 5、 写出RMSE的计算公式，并说明公式中每个变量的含义。（5分）

In [ ]:
RMSE为均方根误差，公式为sqrt(mean((y_r-y_p)^2)),y_r为实际值，y_p为预测值。

#### 6、Deep AutoEncoder 的核心思想是什么。（10分）

In [ ]:
核心的思想是希望能通过反向传播训练网络最小化输入和输出的误差，从而完成对未知数据的预估。

## 二、代码题（60分）

#### 7、请按要求完成基于 tensorflow 的 Deep AutoEncoder 算法。（共30分）

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import tensorflow as tf

#### 加载数据

In [3]:
df = pd.read_csv('ratings.dat', sep='\t', names=['user', 'item', 'rating', 'timestamp'], header=None)

#### (1) 去掉评分数据 df 中的 “timestamp” 这一列，并输出 df 前五行（5分）

In [4]:
del df['timestamp']
df[:5]

,user,item,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


#### 查看用户数和电影数

In [5]:
num_items = df.item.nunique()
num_users = df.user.nunique()
print("USERS: {} ITEMS: {}".format(num_users, num_items))

USERS: 6040 ITEMS: 3706


#### (2) 对 df 的 “rating” 列做 Normalization，展示 df 前 5 行（5分）

In [7]:
from sklearn import preprocessing
r = df['rating'].values.astype('float')
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(r.reshape(-1,1))
df_normalized = pd.DataFrame(x_scaled)
df['rating'] = df_normalized
df[:5]

,user,item,rating
0,1,1193,1.00
1,1,661,0.50
2,1,914,0.50
3,1,3408,0.75
4,1,2355,1.00


#### (3) 把 df 转成 user-item 矩阵，存储在变量 matrix 中 ，用 0 填充矩阵的缺失值，从matrix中取出user5对item6的评分值（5分）

In [11]:
matrix = df.pivot(index='user',columns='item',values='rating')
matrix.fillna(0,inplace=True)
matrix.loc[5,6]

0.25

#### 获取用户和物品列表

In [12]:
users = matrix.index.tolist()
items = matrix.columns.tolist()

#### 将 matrix 从 dataframe 转换成 numpy 数组

In [13]:
matrix = matrix.as_matrix()

#### 网络超参数

In [14]:
num_input = num_items
num_hidden_1 = 10
num_hidden_2 = 5

#### 隐层的变量初始化

In [15]:
weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1], dtype=tf.float64)),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2], dtype=tf.float64)),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1], dtype=tf.float64)),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input], dtype=tf.float64)),
}

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2], dtype=tf.float64)),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'decoder_b2': tf.Variable(tf.random_normal([num_input], dtype=tf.float64)),
}

#### (4) 构建 encoder 和 decoder （5分）

In [16]:
def encoder(x):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
    return layer_2

def decoder(x):
    layer_1 =  tf.nn.relu(tf.add(tf.matmul(x, weights['decoder_h1']), biases['decoder_b1']))
    layer_2 =  tf.nn.relu(tf.add(tf.matmul(layer_1, weights['decoder_h2']), biases['decoder_b2']))
    return layer_2

#### 构建整个模型

In [17]:
X = tf.placeholder(tf.float64, [None, num_input])
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

#### 预测 y 值和真实 y 值

In [18]:
y_pred = decoder_op
y_true = X

#### 定义损失函数和优化器

In [19]:
loss = tf.losses.mean_squared_error(y_true, y_pred)
optimizer = tf.train.RMSPropOptimizer(0.03).minimize(loss)

#### 定义评估准则

In [20]:
eval_x = tf.placeholder(tf.int32, )
eval_y = tf.placeholder(tf.int32, )
pre, pre_op = tf.metrics.precision(labels=eval_x, predictions=eval_y)

#### 变量初始化

In [21]:
init = tf.global_variables_initializer()
local_init = tf.local_variables_initializer()

#### 在 session 中 run

In [22]:
predictions = pd.DataFrame()
with tf.Session() as session:
    epochs = 100
    batch_size = 250

    session.run(init)
    session.run(local_init)

    num_batches = int(matrix.shape[0] / batch_size)
    matrix = np.array_split(matrix, num_batches)

    for i in range(epochs):

        avg_cost = 0

        for batch in matrix:
            _, l = session.run([optimizer, loss], feed_dict={X: batch})
            avg_cost += l

        avg_cost /= num_batches

        print("Epoch: {} Loss: {}".format(i + 1, avg_cost))

    print("Predictions...")

    matrix = np.concatenate(matrix, axis=0)

    preds = session.run(decoder_op, feed_dict={X: matrix})

    predictions = predictions.append(pd.DataFrame(preds))

    predictions = predictions.stack().reset_index(name='rating')
    predictions.columns = ['user', 'item', 'rating']
    predictions['user'] = predictions['user'].map(lambda value: users[value])
    predictions['item'] = predictions['item'].map(lambda value: items[value])

Epoch: 1 Loss: 149.51889538764954
Epoch: 2 Loss: 5.1992259820302325
Epoch: 3 Loss: 1.4357259174187977
Epoch: 4 Loss: 0.6283764677743117
Epoch: 5 Loss: 0.4077149343987306
Epoch: 6 Loss: 0.26733827715118724
Epoch: 7 Loss: 0.11386655705670516
Epoch: 8 Loss: 0.03647922220019003
Epoch: 9 Loss: 0.021631047207241256
Epoch: 10 Loss: 0.020615851002124447
Epoch: 11 Loss: 0.02061576085786025
Epoch: 12 Loss: 0.02062916538367669
Epoch: 13 Loss: 0.02062787883915007
Epoch: 14 Loss: 0.020616445147121947
Epoch: 15 Loss: 0.02061628255372246
Epoch: 16 Loss: 0.02060810449377944
Epoch: 17 Loss: 0.02061281605468442
Epoch: 18 Loss: 0.020610111687953275
Epoch: 19 Loss: 0.020614335623880226
Epoch: 20 Loss: 0.020608811639249325
Epoch: 21 Loss: 0.020604100738031168
Epoch: 22 Loss: 0.020605522169110674
Epoch: 23 Loss: 0.020600884221494198
Epoch: 24 Loss: 0.020605744832816224
Epoch: 25 Loss: 0.020602707867510617
Epoch: 26 Loss: 0.020600740021715563
Epoch: 27 Loss: 0.020593449977847438
Epoch: 28 Loss: 0.02059919775

#### (5) 为用户42计算top10的推荐结果（10分）
- top10 的筛选依据是评分预测最高的 top10
- 要求推荐的 top10 是该用户没有看过的电影
- 最后输出推荐的 top10 电影的 itemid

In [48]:
df_42p = predictions[predictions.user==42]
df_42r = df[df.user==42]
df_42 = df_42p.append(df_42r)
df_42.drop_duplicates(subset=['item'],keep=False,inplace=True)
df_42.sort_values(by='rating',ascending=False)['item'][:10]

153431    1617
151946       1
151995      50
152798     913
153005    1136
153101    1247
153113    1259
154721    2987
153085    1230
154532    2791
Name: item, dtype: int64

#### 8、请你实现基于Keras的协同深度学习算法。（共30分）

In [51]:
import numpy as np
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from keras.layers import Input, Embedding, Dot, Flatten, Dense, Dropout, Lambda, Add
from keras.layers.noise import GaussianNoise
from keras.initializers import RandomUniform, RandomNormal
from keras.models import Model
from keras.regularizers import l2
from keras import optimizers
from keras import backend as K
from keras.engine.topology import Layer

Using TensorFlow backend.


#### (1) 请你完成 movie_map 这个函数（5分）
- 功能：使用自然数对 movies.dat 中的 movieid 进行重编码
- 返回值：movieid 到编码 id 的字典
- 测试：计算编码字典，存入变量 d（后面有用），输出 d[520]

In [52]:
def movie_map(file='movies.dat'):
    movies = read_csv(file, sep='::', header=None)
    value = movies[0].unique()
    index = range(movies[0].nunique())
    return dict(zip(value,index))
                
d = movie_map()
d[520]

516

#### (2) 请你完成 read_ratings 这个函数（5分）
- 要求返回数据类型为 float32 的二维 numpy 数组：每一行表示用户评分记录 [user,item,rating]
- 原始数据中的 itemid 需要使用编码字典 d 进行重编码
- rating 值需要进行幅度缩放，缩放区间为[0,1]
- 测试：调用 read_ratings ,将返回值存入变量 ratings （后面有用），并输出 ratings[:3]

In [68]:
def read_ratings(file='ratings.dat'):
   
    rating_mat = list()
    with open(file) as fp:
        for line in fp:
            line = line.strip().split('\t')
            user, item, rating = line[0], d[int(line[1])], line[2]
            rating_mat.append( [user, item, rating] )
    min_r = int(min([x[2] for x in rating_mat]))
    max_r = int(max([x[2] for x in rating_mat]))
    for i in range(len(rating_mat)):
        rating_mat[i][2] = (int(rating_mat[i][2])-min_r)/(max_r-min_r)
    return np.array(rating_mat).astype('float32')

ratings = read_ratings()
ratings[:3]

array([[1.000e+00, 1.176e+03, 1.000e+00],
       [1.000e+00, 6.550e+02, 5.000e-01],
       [1.000e+00, 9.020e+02, 5.000e-01]], dtype=float32)

#### (3) 请你完成 train_test_split 这个函数（5分）
- 功能：将评分数据 ratings 划分为训练集和测试集两部分
- 输入：函数 read_ratings 的返回值 ratings
- 输出：训练集 train_mat，测试集 test_mat
- 要求：训练集占 80%，测试集占 20%
- 测试：调用该函数，返回值存入 train_mat, test_mat （后面有用），并展示训练集和测试集的样本数量

In [69]:
def train_test_split(ratings):
    num_train = int(len(ratings)*0.8)
    train_mat = ratings[:num_train]
    test_mat = ratings[num_train:]
    return train_mat,test_mat

train_mat, test_mat = train_test_split(ratings)
train_mat.shape[0], test_mat.shape[0]

(800167, 200042)

#### (4) 请你完成 read_item 这个函数（5分）
- 功能：将 movies.dat 的三列特征进行重编码
- 要求：
    - 第 1 列特征使用编码字典 d 进行编码
    - 第 2、3 列特征使用 LabelEncoder() 编码
    - 返回值有两个，第一个是二维 numpy 数组 ，也就是一个矩阵：每一行对应原始数据 movies.dat 的一行记录；第二个是特征维度，也就是矩阵的列数
    - 调用该函数，返回值存入 item_mat,item_feat_dim （后面有用），打印 item_mat[:3] 和 item_feat_dim

In [72]:
def read_item(file='movies.dat'):
    
    item = read_csv(file, sep='::', header=None)
    item[0] = item[0].apply(lambda x:d[int(x)])
    for i in range(1,3):
        item[i] = LabelEncoder().fit_transform(item[i])
    return np.array(item),item.shape[1]

item_mat,item_feat_dim = read_item()
print(item_mat[:3])
print(item_feat_dim)

[[   0 3574  145]
 [   1 1858  115]
 [   2 1483  207]]
3


####  CollaborativeDeepLearning 实现

In [73]:
class CollaborativeDeepLearning:
    
    def __init__(self, item_mat, hidden_layers):
        '''
        hidden_layers = a list of three integer indicating the embedding dimension of autoencoder
        item_mat = item feature matrix with shape (# of item, # of item features)
        '''
        assert(len(hidden_layers)==3)
        self.item_mat = item_mat
        self.hidden_layers = hidden_layers
        self.item_dim = hidden_layers[0]
        self.embedding_dim = hidden_layers[-1]
        
    def pretrain(self, lamda_w=0.1, encoder_noise=0.1, dropout_rate=0.1, activation='sigmoid', batch_size=64, epochs=10):
        '''
        layer-wise pretraining on item features (item_mat)
        '''
        self.trained_encoders = []
        self.trained_decoders = []
        X_train = self.item_mat
        
        for input_dim, hidden_dim in zip(self.hidden_layers[:-1], self.hidden_layers[1:]):
            
            pretrain_input = Input(shape=(input_dim,))
            encoded = GaussianNoise(stddev=encoder_noise)(pretrain_input)
            encoded = Dropout(dropout_rate)(encoded)
            encoder = Dense(hidden_dim, activation=activation, kernel_regularizer=l2(lamda_w), bias_regularizer=l2(lamda_w))(encoded)
            decoder = Dense(input_dim, activation=activation, kernel_regularizer=l2(lamda_w), bias_regularizer=l2(lamda_w))(encoder)
            
            # autoencoder
            ae = Model(inputs=pretrain_input, outputs=decoder)
            
            # encoder
            ae_encoder = Model(inputs=pretrain_input, outputs=encoder)
            
            # decoder
            encoded_input = Input(shape=(hidden_dim,))
            
            decoder_layer = ae.layers[-1] # the last layer
            ae_decoder = Model(encoded_input, decoder_layer(encoded_input))

            ae.compile(loss='mse', optimizer='rmsprop')
            ae.fit(X_train, X_train, batch_size=batch_size, epochs=epochs, verbose=2)

            self.trained_encoders.append(ae_encoder)
            self.trained_decoders.append(ae_decoder)
            X_train = ae_encoder.predict(X_train)

    def fineture(self, train_mat, test_mat, lamda_u=0.1, lamda_v=0.1, lamda_n=0.1, lr=0.001, batch_size=64, epochs=10):
        '''
        Fine-tuning with rating prediction
        '''
        num_user = int( max(train_mat[:,0].max(), test_mat[:,0].max()) + 1 )
        num_item = int( max(train_mat[:,1].max(), test_mat[:,1].max()) + 1 )

        # item autoencoder 
        itemfeat_InputLayer = Input(shape=(self.item_dim,), name='item_feat_input')
        encoded = self.trained_encoders[0](itemfeat_InputLayer)
        encoded = self.trained_encoders[1](encoded)
        decoded = self.trained_decoders[1](encoded)
        decoded = self.trained_decoders[0](decoded)

        # user embedding
        user_InputLayer = Input(shape=(1,), dtype='int32', name='user_input')
        user_EmbeddingLayer = Embedding(input_dim=num_user, output_dim=self.embedding_dim,
                                        input_length=1, name='user_embedding', 
                                        embeddings_regularizer=l2(lamda_u),
                                        embeddings_initializer=RandomNormal(mean=0, stddev=1))(user_InputLayer)
        user_EmbeddingLayer = Flatten(name='user_flatten')(user_EmbeddingLayer)

        # item embedding
        item_InputLayer = Input(shape=(1,), dtype='int32', name='item_input')
        item_OffsetVector = Embedding(input_dim=num_item, output_dim=self.embedding_dim, 
                                      input_length=1, name='item_offset_vector',
                                      embeddings_regularizer=l2(lamda_v), 
                                      embeddings_initializer=RandomNormal(mean=0, stddev=1))(item_InputLayer)
        item_OffsetVector = Flatten(name='item_flatten')(item_OffsetVector)
        item_EmbeddingLayer = Add()([encoded, item_OffsetVector]) 
        
        # rating prediction
        dotLayer = Dot(axes = -1, name='dot_layer')([user_EmbeddingLayer, item_EmbeddingLayer])

        self.cdl_model = Model(inputs=[user_InputLayer, item_InputLayer, itemfeat_InputLayer], outputs=[dotLayer, decoded])
        self.cdl_model.compile(optimizer='rmsprop', loss=['mse', 'mse'], loss_weights=[1, lamda_n])

        train_user, train_item, train_item_feat, train_label = self.matrix2input(train_mat)
        test_user, test_item, test_item_feat, test_label = self.matrix2input(test_mat)

        model_history = self.cdl_model.fit([train_user, train_item, train_item_feat], 
                                           [train_label, train_item_feat], 
                                           epochs=epochs, batch_size=batch_size, 
                                           validation_data=([test_user, test_item, test_item_feat], [test_label, test_item_feat]))
        return model_history

    def matrix2input(self, rating_mat):
        train_user = rating_mat[:, 0].reshape(-1, 1).astype(int)
        train_item = rating_mat[:, 1].reshape(-1, 1).astype(int)
        train_label = rating_mat[:, 2].reshape(-1, 1)
        train_item_feat = [self.item_mat[train_item[x]][0] for x in range(train_item.shape[0])]
        return train_user, train_item, np.array(train_item_feat), train_label
    
    def build(self):
        # rating prediction
        prediction_layer = Dot(axes = -1, name='prediction_layer')([user_EmbeddingLayer, encoded])
        self.model = Model(inputs=[user_InputLayer, itemfeat_InputLayer], outputs=[prediction_layer])
        
    def getRMSE(self, test_mat):
        test_user, test_item, test_item_feat, test_label = self.matrix2input(test_mat)
        pred_out = self.cdl_model.predict([test_user, test_item, test_item_feat])
        return np.sqrt(np.mean(np.square(test_label.flatten() - pred_out[0].flatten())))

#### (5) 成功运行下面的代码，输出RMSE（10分）

In [74]:
# To Run
model = CollaborativeDeepLearning(item_mat, [item_feat_dim, 16, 8])
model.pretrain(lamda_w=0.001, encoder_noise=0.3, epochs=10)
model_history = model.fineture(train_mat, test_mat, lamda_u=0.01, lamda_v=0.1, lamda_n=0.1, lr=0.01, epochs=4)
model.getRMSE(test_mat)

Epoch 1/10
 - 0s - loss: 3362810.8983
Epoch 2/10
 - 0s - loss: 3362529.0643
Epoch 3/10
 - 0s - loss: 3362253.1535
Epoch 4/10
 - 0s - loss: 3362099.5319
Epoch 5/10
 - 0s - loss: 3361980.8258
Epoch 6/10
 - 0s - loss: 3361893.0852
Epoch 7/10
 - 0s - loss: 3361823.1601
Epoch 8/10
 - 0s - loss: 3361759.6746
Epoch 9/10
 - 0s - loss: 3361716.1151
Epoch 10/10
 - 0s - loss: 3361683.5159
Epoch 1/10
 - 0s - loss: 0.2509
Epoch 2/10
 - 0s - loss: 0.1983
Epoch 3/10
 - 0s - loss: 0.1598
Epoch 4/10
 - 0s - loss: 0.1334
Epoch 5/10
 - 0s - loss: 0.1166
Epoch 6/10
 - 0s - loss: 0.1066
Epoch 7/10
 - 0s - loss: 0.1010
Epoch 8/10
 - 0s - loss: 0.0976
Epoch 9/10
 - 0s - loss: 0.0954
Epoch 10/10
 - 0s - loss: 0.0939
Train on 800167 samples, validate on 200042 samples
Epoch 1/4
800167/800167 [==============================] - 38s 47us/step - loss: 320009.9658 - dot_layer_loss: 0.6862 - model_3_loss: 3198583.5481 - val_loss: 311579.5032 - val_dot_layer_loss: 0.5071 - val_model_3_loss: 3115789.5770
Epoch 2/4
800

0.71212685